In [1]:
import os
import glob
import pickle
import pandas as pd
from tqdm.notebook import tqdm

data_dir = "data"
xls_dir = "xls"

raw_prefix = "test"
processed_prefix = "ind"


xls_paths = glob.glob(os.path.join(xls_dir, "*.xls"))

os.makedirs(data_dir, exist_ok=True)

raw = dict()
processed = dict()

for i, file_path in enumerate(tqdm(xls_paths, desc="Files")):
    xls_file = pd.ExcelFile(file_path)
    xls_file_id = int(file_path.split('.')[0][-1])
    sheet_names = [sheet_name for sheet_name in xls_file.sheet_names if sheet_name.lower().startswith((raw_prefix, processed_prefix))]
    for sheet_name in tqdm(sheet_names, desc="Sheets", leave=False):
        try:
            sheet_id = int(sheet_name[-3:])
            id = f'{(sheet_id + 50 * (xls_file_id - 1)):03d}'
            if sheet_name.lower().startswith(raw_prefix):
                sample_df = pd.read_excel(file_path, sheet_name=sheet_name, header=0, skiprows=[1])
                data = sample_df.iloc[1:].reset_index(drop=True)
                raw[id] = data
            elif sheet_name.lower().startswith(processed_prefix):
                sample_df = pd.read_excel(file_path, sheet_name=sheet_name, header=0)
                #info = sample_df.iloc[0, -11:].to_frame().T
                data = sample_df.iloc[1:].reset_index(drop=True)
                processed[id] = data
        except ValueError:
            print(f"Skipping sheet {sheet_name} in file {file_path}")
            pass


with open(f'{data_dir}/processed.pkl', 'wb') as f:
    pickle.dump(processed, f)

with open(f'{data_dir}/raw.pkl', 'wb') as f:
    pickle.dump(raw, f)

Files:   0%|          | 0/4 [00:00<?, ?it/s]

Sheets:   0%|          | 0/98 [00:00<?, ?it/s]

049
048
047
046
045
044
043
042
041
040
039
038
037
036
035
034
033
032
031
030
029
028
027
026
025
024
023
022
021
020
019
018
017
016
015
014
013
012
011
010
009
008
007
006
005
004
003
002
001
001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049


Sheets:   0%|          | 0/97 [00:00<?, ?it/s]

099
098
097
096
095
094
093
092
091
090
089
088
087
086
085
084
083
082
081
080
079
078
077
076
075
074
073
072
071
070
069
068
067
066
065
064
063
062
061
060
059
058
057
056
055
054
053
052
051
051
052
053
054
055
056
057
058
059
060
061
062
063
064
065
066
067
068
070
071
072
073
074
075
076
077
078
079
080
081
082
083
084
085
086
087
088
089
090
091
092
093
094
095
096
097
098
099


Sheets:   0%|          | 0/98 [00:00<?, ?it/s]

199
198
197
196
195
194
193
192
191
190
189
188
187
186
185
184
183
182
181
180
179
178
177
176
175
174
173
172
171
170
169
168
167
166
165
164
163
162
161
160
159
158
157
156
155
154
153
152
151
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


Sheets:   0%|          | 0/98 [00:00<?, ?it/s]

149
148
147
146
145
144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
